In [63]:
import duckdb as ddb
import pandas as pd

Establish a connection to the DuckDB database file and

con  # Display the connection object to verify it is active

In [64]:
con = ddb.connect("../air_quality.db")
con

In [65]:
df = con.query("select * from raw.air_quality where parameter in ('so2', 'pm10', 'pm25')").to_df()
#len(df)
df.head()

,location_id,sensors_id,location,datetime,lat,lon,parameter,units,value,month,year,ingestion_datetime
0,225405,1303524,Bongani Mabaso Eco Park-237701,2024-02-29 23:00:00,-26.77761,27.837328,pm25,µg/m³,22.095,03,2024,2026-01-25 00:05:00.764864
1,225405,1303524,Bongani Mabaso Eco Park-237701,2024-03-01 00:00:00,-26.77761,27.837328,pm25,µg/m³,23.414,03,2024,2026-01-25 00:05:00.764864
2,225405,1303524,Bongani Mabaso Eco Park-237701,2024-03-01 01:00:00,-26.77761,27.837328,pm25,µg/m³,25.818,03,2024,2026-01-25 00:05:00.764864
3,225405,1303524,Bongani Mabaso Eco Park-237701,2024-03-01 02:00:00,-26.77761,27.837328,pm25,µg/m³,30.793,03,2024,2026-01-25 00:05:00.764864
4,225405,1303524,Bongani Mabaso Eco Park-237701,2024-03-01 03:00:00,-26.77761,27.837328,pm25,µg/m³,36.194,03,2024,2026-01-25 00:05:00.764864


In [66]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49992 entries, 0 to 49991
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   location_id         49992 non-null  int64         
 1   sensors_id          49992 non-null  int64         
 2   location            49992 non-null  object        
 3   datetime            49992 non-null  datetime64[us]
 4   lat                 49992 non-null  float64       
 5   lon                 49992 non-null  float64       
 6   parameter           49992 non-null  object        
 7   units               49992 non-null  object        
 8   value               49992 non-null  float64       
 9   month               49992 non-null  object        
 10  year                49992 non-null  int64         
 11  ingestion_datetime  49992 non-null  datetime64[us]
dtypes: datetime64[us](2), float64(3), int64(3), object(4)
memory usage: 4.6+ MB


In [67]:
df.describe()

,location_id,sensors_id,datetime,lat,lon,value,year,ingestion_datetime
count,49992.000000,4.999200e+04,49992,49992.000000,49992.000000,49992.000000,49992.0,49992
mean,225434.746459,1.303496e+06,2024-02-07 07:16:11.795487,-28.435041,28.081450,16.288907,2024.0,2026-01-25 12:08:15.821628
min,225405.000000,1.303377e+06,2024-01-01 01:00:00,-29.637583,26.241952,0.000000,2024.0,2026-01-25 00:05:00.764864
25%,225428.000000,1.303432e+06,2024-01-21 01:00:00,-29.637583,26.241952,0.008098,2024.0,2026-01-25 14:00:59.818650
50%,225443.000000,1.303454e+06,2024-02-06 19:00:00,-29.138507,27.867111,6.720000,2024.0,2026-01-25 14:17:52.350280
75%,225443.000000,1.303602e+06,2024-02-25 15:00:00,-26.777610,30.337389,20.263000,2024.0,2026-01-25 15:23:46.120435
max,225449.000000,1.303636e+06,2024-03-12 10:00:00,-26.479639,30.337389,875.556000,2024.0,2026-01-25 15:34:50.091009
std,13.874130,8.776106e+01,NaN,1.336016,1.672191,37.782528,0.0,NaN


In [68]:
df.describe(include='O')

,location,parameter,units,month
count,49992,49992,49992,49992
unique,4,3,2,3
top,Pelonomi-NAQI-237739,so2,µg/m³,01
freq,17300,18439,31553,21165


In [69]:
df[df.duplicated(subset=["location_id","datetime", "parameter", "units", "value", "month", "year", "ingestion_datetime"])]

,location_id,sensors_id,location,datetime,lat,lon,parameter,units,value,month,year,ingestion_datetime


In [70]:
df.groupby(by=["parameter", "location_id"], as_index=False).count()

,parameter,location_id,sensors_id,location,datetime,lat,lon,units,value,month,year,ingestion_datetime
0,pm10,225405,2099,2099,2099,2099,2099,2099,2099,2099,2099,2099
1,pm10,225428,5265,5265,5265,5265,5265,5265,5265,5265,5265,5265
2,pm10,225443,5805,5805,5805,5805,5805,5805,5805,5805,5805,5805
3,pm10,225449,5185,5185,5185,5185,5185,5185,5185,5185,5185,5185
4,pm25,225405,2099,2099,2099,2099,2099,2099,2099,2099,2099,2099
5,pm25,225428,5265,5265,5265,5265,5265,5265,5265,5265,5265,5265
6,pm25,225443,5835,5835,5835,5835,5835,5835,5835,5835,5835,5835
7,so2,225405,2099,2099,2099,2099,2099,2099,2099,2099,2099,2099
8,so2,225428,5265,5265,5265,5265,5265,5265,5265,5265,5265,5265
9,so2,225443,5660,5660,5660,5660,5660,5660,5660,5660,5660,5660


In [71]:
df[df['value'] < 0 ]

,location_id,sensors_id,location,datetime,lat,lon,parameter,units,value,month,year,ingestion_datetime


In [72]:
con.close()